In [1]:
# import basic modules here 
import numpy as np
import cv2
import os 
import glob
import time
import sys
import json
import slidingwindow as sw
import matplotlib.pyplot as plt 
import pycocotools.mask as maskUtils
import mmcv 
import torch
from pycocotools.coco import COCO
from pycocotools import coco
from IPython.display import clear_output, display

# import mmdet-related modules 
from mmdet.apis import init_detector, inference_detector
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from scipy.ndimage import binary_fill_holes

# import shm_tools here
sys.path.append('..')
from shm_tools.shm_utils import imread, imwrite


In [ ]:

path = '/home/soojin/mmdetection/data/concrete_damage_autocon/test2018/'
im_list = glob.glob(os.path.join(path,'*.jpg')) + glob.glob(os.path.join(path,'*.JPG'))

config = '/home/soojin/mmdetection/project_work_dirs/multiple_damage_detection/configs/bilin_112/mask_rcnn_r101_fpn_1x_mask_bilin_112_overlap.py'
checkpoint = '/home/soojin/mmdetection/project_work_dirs/multiple_damage_detection/work_dirs/mask_rcnn_r101_fpn_1x_mask_bilin_112_overlap/epoch_42.pth'
device = 'cuda:0'
model = init_detector(config, checkpoint, device=device)

intersection_meter = AverageMeter()
union_meter = AverageMeter()
target_meter = AverageMeter()

intersection_list = []
union_list = []
target_list = []

for img_num , img_path in enumerate(im_list): 
    clear_output(wait=True)
    print('Image No.'+ str(img_num+1) + 'is under process')
    img_name = os.path.basename(img_path)   
    image = imread(img_path)
    result = inference_detector(model, img_path)

    num_class = 5 # background + object classes 
    bbox_result, segm_result = result

    for segm_result_ in segm_result : 
        if segm_result_:
            img_size = segm_result_[0]['size']
    seg_sum = np.zeros((img_size[0], img_size[1], num_class), dtype=np.float32)
    mask_sum = np.zeros((img_size[0], img_size[1]) , dtype=np.bool)

    for class_, bboxes in enumerate(bbox_result):

        class_idx = class_ + 1 

        for idx, bbox in enumerate(bboxes) : 
            confidence = bbox[-1]
            if confidence >= 0.5 : 
                mask = maskUtils.decode(segm_result[class_][idx]) 
                mask_sum = mask_sum  + mask
                seg_sum[:, :, class_idx] += mask

    seg_sum[:,:, 0] = mask_sum == 0
    seg_sum[seg_sum > 1] = 1
    seg_2d = np.zeros((image.shape[0], image.shape[1]), dtype = np.uint8)
    seg_2d[seg_sum[:, :, 2] == 1] = 2 # efflorescence _non_overlap
    seg_2d[seg_sum[:, :, 4] == 1] = 4 # spalling 
    seg_2d[seg_sum[:, :, 3] == 1] = 3 # rebar exposure 
    seg_2d[seg_sum[:, :, 1] == 1] = 1 # crack 

    fast_inference = False
    annot_path = os.path.join(path, 'annot')
    annot_img_list = glob.glob(os.path.join(annot_path, os.path.splitext(img_name)[0]+ '*.jpg'))

    lower_thr = 150
    upper_thr = 200

    COLOR_OPTIONS = {
        'red'    :  [ '>', upper_thr,  '<', lower_thr, '<', lower_thr],
        'green'  : [ '<', lower_thr,  '>', upper_thr, '<', lower_thr],
    #     'blue'   :  [ '<', lower_thr,  '<', lower_thr, '>', upper_thr],
        'cyan'   : [ '<', 150,  '>', 200, '>', 200],
        'yellow' :  [ '>', 200,  '>', 200, '<', 150]
    }


    damage_areas = np.zeros((image.shape[0], image.shape[1], 4), dtype = np.uint8)

    for annot_img_path in annot_img_list:

        annot_img = imread(annot_img_path)

        for color_num, color in enumerate(COLOR_OPTIONS) : 

            rgb_thr = COLOR_OPTIONS[color]
            R = comparison_operator(annot_img[:, :, 0], rgb_thr[1], rgb_thr[0])
            G = comparison_operator(annot_img[:, :, 1], rgb_thr[3], rgb_thr[2])
            B = comparison_operator(annot_img[:, :, 2], rgb_thr[5], rgb_thr[4])

            damage_area = (R & G & B)
            damage_area = np.asarray(damage_area, dtype = np.uint8)
            damage_area = binary_fill_holes(damage_area).astype('uint8')
            damage_areas[:, :, color_num] = damage_areas[:, :, color_num] + damage_area

    damage_areas[damage_areas > 1 ] = 1

    seg_gt_2d = np.zeros((annot_img.shape[0], annot_img.shape[1]), dtype = np.uint8)

    seg_gt_2d[damage_areas[:, :, 3] == 1] = 4 # spalling 
    seg_gt_2d[damage_areas[:, :, 1] == 1] = 2 # efflorescence 
    seg_gt_2d[damage_areas[:, :, 2] == 1] = 3 # rebar exposure 
    seg_gt_2d[damage_areas[:, :, 0] == 1] = 1 # crack 


    intersection, union, target = intersectionAndUnion(seg_2d, seg_gt_2d, 5)
    intersection_list.append(intersection)
    union_list.append(union)
    target_list.append(target)
    intersection_meter.update(intersection[1:])
    union_meter.update(union[1:])
    target_meter.update(target[1:])
    accuracy = sum(intersection_meter.val) / (sum(target_meter.val) + 1e-10)

iou_class = intersection_meter.sum / (union_meter.sum + 1e-10)
accuracy_class = intersection_meter.sum / (target_meter.sum + 1e-10)
mIoU = np.mean(iou_class)
mAcc = np.mean(accuracy_class)
allAcc = sum(intersection_meter.sum) / (sum(target_meter.sum) + 1e-10)

print('mIoU:', mIoU, 'mAcc:', mAcc, 'allAcc:', allAcc)